In [ ]:
# === Notebook bootstrap: make repo root importable ===
import sys
from pathlib import Path

_cwd = Path.cwd().resolve()
for p in [_cwd, *_cwd.parents]:
    if (p / "src").exists():
        if str(p) not in sys.path:
            sys.path.insert(0, str(p))
        break

print("cwd:", _cwd)
print("sys.path[0]:", sys.path[0])


In [ ]:
#01
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from src.io_utils import load_pickle, outputs_dir
from src.features import build_log_ad_from_spend
from src.modeling import build_X_for_model, predict_annual_sales_52

OUT_DIR = outputs_dir()

WS_PATH = Path(OUT_DIR) / "weekly_score.pkl"
MODEL_PATH = Path(OUT_DIR) / "final_model.pkl"
DF_PATH = Path(OUT_DIR) / "df_w_feat.pkl"

ws = load_pickle(WS_PATH).copy()
final_model = load_pickle(MODEL_PATH)
df_w = load_pickle(DF_PATH).copy()  # ← df_w_feat を df_w として扱う


# Compatibility key (used only for ws merge / weighting)
if "week_of_year" in df_w.columns:
    df_w["week_of_year"] = df_w["week_of_year"].astype(int)
elif "iso_week" in df_w.columns:
    df_w["week_of_year"] = df_w["iso_week"].astype(int)
else:
    df_w["week_of_year"] = df_w["Week"].dt.isocalendar().week.astype(int)


In [ ]:
#02
# =========================
# As-Is annual budget summary (for plotting + optimization)
# =========================

# 色固定（#03で使用）
COLOR_MAP = {
    "online_spend": "#8453F6",
    "broadcast_spend": "#4DA9CD",
    "ooh_print_spend": "#F1CD46",
}
MEDIA_COLS = ["broadcast_spend", "ooh_print_spend", "online_spend"]

# FYは df_w_feat 側で正規化済みの fy_year を使う
df_tmp = df_w.copy()
df_tmp["FY"] = df_tmp["fy_year"].astype(int)

# FYごとの年次合計 → 年平均（As-Is）
df_year = (
    df_tmp.groupby("FY", as_index=False)[["sales", "total_spend"] + MEDIA_COLS]
    .sum()
    .sort_values("FY")
)

avg_series = df_year.drop(columns=["FY"]).mean()
avg_df = (
    avg_series.reset_index()
    .rename(columns={"index": "metric", 0: "avg_value"})
)

display(df_year)
display(avg_df)




In [ ]:
#03
# As-Is可視化
# 媒体別平均投資額（棒）
media_df = avg_df[avg_df["metric"].isin(MEDIA_COLS)].copy()
media_df["color"] = media_df["metric"].map(COLOR_MAP)

plt.figure(figsize=(8, 5))
plt.bar(media_df["metric"], media_df["avg_value"], color=media_df["color"])
plt.title("Average Spend by Media (As-Is)")
plt.xlabel("Media")
plt.ylabel("Average Annual Spend Amount")
plt.xticks(rotation=20, fontsize=12)
plt.tight_layout()
plt.show()

# 構成比（円）
total_spend = float(avg_df.loc[avg_df["metric"] == "total_spend", "avg_value"].iloc[0])
media_df["share_pct"] = media_df["avg_value"] / total_spend * 100

plt.figure(figsize=(7, 7))
plt.pie(
    media_df["share_pct"],
    labels=media_df["metric"],
    colors=media_df["metric"].map(COLOR_MAP),
    autopct="%1.1f%%",
    textprops={"fontsize": 12},
)
plt.title("Media Spend Share of Total Spend (As-Is")
plt.tight_layout()
plt.show()

display(media_df[["metric", "avg_value", "share_pct"]].round(2))


In [ ]:
#04
# 代表年（FY2022）で 52週の df_sim を作る（ISO週テンプレ）
# 用途：配分・シミュレーション用の週テンプレート

FY_REP = 2022

tmp = df_w[df_w["fy_year"] == FY_REP].copy()

# ISO週をテンプレキーにする（1..52）
tmp["week_of_year"] = tmp["iso_week"]
tmp.loc[tmp["week_of_year"] == 53, "week_of_year"] = 52

df_sim = (
    tmp.sort_values("Week")
       .drop_duplicates("week_of_year")
       .query("1 <= week_of_year <= 52")
       .reset_index(drop=True)
       .copy()
)

if len(df_sim) != 52:
    raise ValueError(f"FY_REP={FY_REP} cannot build 52-week template by iso_week. len={len(df_sim)}")

display(df_sim[["Week", "fy_year", "fy_week", "iso_week", "week_of_year"]].head())


In [ ]:
#05
# Online の週重み（ROI×需要）を作って付与
ws_tmp = ws.copy().query("1 <= week_of_year <= 52").copy()  # ★52週テンプレに合わせる

# 既に score_online があれば正本を使う（なければここで計算）
if "score_online" not in ws_tmp.columns:
    ws_tmp["score_online"] = ws_tmp["baseline_hat"] * ws_tmp["roi_online"].clip(lower=0)

den = float(ws_tmp["score_online"].sum())
if den <= 0:
    raise ValueError("score_online の合計が 0 以下です。")

ws_tmp["online_weight"] = ws_tmp["score_online"] / den

df_sim = df_sim.merge(
    ws_tmp[["week_of_year", "online_weight"]],
    on="week_of_year",
    how="left",
    validate="1:1"
)

# 型を安定化
df_sim["online_weight"] = df_sim["online_weight"].astype(float)

# 欠損チェック（ここは残す：後で静かに壊れるのを防ぐ）
if df_sim["online_weight"].isna().any():
    miss = sorted(df_sim.loc[df_sim["online_weight"].isna(), "week_of_year"].tolist())
    raise ValueError(f"online_weight 未付与の週があります: {miss}")

# 正規化チェック（ここも残す）
print("sum(online_weight) =", df_sim["online_weight"].sum())


In [ ]:
#06

EXOG = list(final_model.model.exog_names)

# decay は 04 で確定した best_decay を正として読む
BEST_DECAY_PATH = Path(OUT_DIR) / "best_decay.pkl"
best_decay = load_pickle(BEST_DECAY_PATH)
D_ONLINE = float(best_decay["d_online"])
D_BROADCAST = float(best_decay["d_broadcast"])

# 年間平均 total_spend（As-Is）
total_budget = float(avg_df.loc[avg_df["metric"] == "total_spend", "avg_value"].iloc[0])
print("annual total_budget =", total_budget)

# Broadcast pulse weeks (ISO week-based)
PULSE_WEEKS = [16, 20]
mask = df_sim["week_of_year"].isin(PULSE_WEEKS).values
pulse_n = int(mask.sum())
if pulse_n <= 0:
    raise ValueError(f"PULSE_WEEKS not found in df_sim: {PULSE_WEEKS}")

online_share_grid = np.arange(0.2, 0.81, 0.05)
rows = []

for s in online_share_grid:
    annual_budget_online = total_budget * float(s)
    annual_budget_broadcast = total_budget * float(1 - s)

    # Online: weight配分
    online_path = annual_budget_online * df_sim["online_weight"].values

    # Broadcast: pulse配分
    broadcast_path = np.zeros(len(df_sim), dtype=float)
    broadcast_path[mask] = annual_budget_broadcast / pulse_n


    df_tmp = df_sim.copy()
    df_tmp["log_ad_online"] = build_log_ad_from_spend(online_path, D_ONLINE)
    df_tmp["log_ad_broadcast"] = build_log_ad_from_spend(broadcast_path, D_BROADCAST)
    df_tmp["log_ad_ooh"] = 0.0

    X_52 = build_X_for_model(df_tmp, EXOG)
    sales_hat = float(predict_annual_sales_52(final_model, X_52))
    # -------------------------------------------

    rows.append({
        "online_share": float(s),
        "broadcast_share": float(1 - s),
        "annual_sales_hat": sales_hat
    })

res_52 = pd.DataFrame(rows).sort_values("online_share").reset_index(drop=True)
display(res_52)

In [ ]:
# 07
from matplotlib.ticker import PercentFormatter
# ================================
# Best mix & visualization 
# ================================
if res_52.empty:
    raise ValueError("res_52 is empty. Check the sweep loop output.")

best = res_52.loc[res_52["annual_sales_hat"].idxmax()]

print(
    f"Best mix (52-week template): "
    f"Online {best['online_share']:.0%}, "
    f"Broadcast {best['broadcast_share']:.0%}, "
    f"Predicted annual sales {best['annual_sales_hat']:,.0f}"
)

fig, ax = plt.subplots(figsize=(8, 4))

ax.plot(
    res_52["online_share"],
    res_52["annual_sales_hat"],
    marker="o",
    linewidth=2,
    color="#A6CE25",
)

ax.scatter(
    best["online_share"],
    best["annual_sales_hat"],
    s=100,
    zorder=5,
    color="#DB544E",
)

ax.axvline(
    best["online_share"],
    linestyle="--",
    linewidth=1.5,
    color="#DB544E",
)

# --- X軸（Online share）---
ax.set_xlabel("Online share")
ax.xaxis.set_major_formatter(PercentFormatter(xmax=1.0))

# --- y軸（sales）---
ax.set_ylabel("Predicted annual sales")
ax.set_title("Predicted annual sales vs media mix (52-week template)")
ax.grid(axis="y")

plt.tight_layout()
plt.show()


In [ ]:
# 保存：52週テンプレ（FY2022）Online vs Broadcast 配分カーブ
# NOTE: average annual budget (As-Is)
csv_path = OUT_DIR / "df_alloc_curve.csv"
res_52.to_csv(csv_path, index=False)

print("Saved:", csv_path)


# NOTE:
- This simulation evaluates annual budget allocation using a representative 52-week fiscal year (FY2022), assuming stable seasonality and marginal response.